In [ ]:
# 1. Устанавливаем библиотеки
# pip install transformers datasets wandb huggingface_hub
!pip install datasets
#!pip install -U transformers
!pip install -U transformers datasets wandb huggingface_hub
!pip install fsspec==2025.3.2
!pip install nvidia-cublas-cu12==12.4.5.8 nvidia-cuda-cupti-cu12==12.4.127 nvidia-cuda-runtime-cu12==12.4.127
import time

from huggingface_hub import login
import os
os.environ['HF_TOKEN'] = 'hf_***'


import os
import torch
from datasets import load_dataset
from transformers import (
    GPT2Tokenizer,
    GPT2Config,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
)
import wandb
from huggingface_hub import login

# 2. Вход в HuggingFace и Weights & Biases
login(token="hf_***")  # <== сюда свой токен Hugging Face
wandb.login(key="f3b0224b314359387370d2f4d2f6a5763c894caa")  # <== сюда свой токен wandb

# 3. Загрузка tiny_shakespeare датасета
# Laden Sie die Datei herunter und speichern Sie sie/Скачиваем и сохраняем файл
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O tinyshakespeare.txt

#url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
#response = requests.get(url)
#text = response.text
#from datasets import load_dataset

#dataset = load_dataset('text', data_files={'train': 'tinyshakespeare.txt'})
dataset = load_dataset('text', data_files={'train': 'tinyshakespeare.txt'})
dataset["train"] = dataset["train"].select(range(3000))  # <= только3000 1000 примеров


#dataset = load_dataset('url')  # либо можно локально скачать

# 4. Токенизация
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # важно для паддинга

# def tokenize_function(examples):
#     return tokenizer(examples["text"], return_special_tokens_mask=True)
def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"],
        padding="max_length",     # добавляем паддинг
        truncation=True,          # обрезаем слишком длинные тексты
        max_length=64,                #256,           # фиксируем длину (как в config)
        return_special_tokens_mask=True,
        return_attention_mask=True  # Указание маски внимания
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# 5. Разбиваем на тренировочный и валидационный набор
tokenized_datasets = tokenized_datasets["train"].train_test_split(test_size=0.1)
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

# 6. Конфигурация маленькой GPT-2 модели
config = GPT2Config(
    vocab_size=len(tokenizer),
    n_positions=128,                        #256,
    n_ctx=128,                              #256,
    n_embd=128,                   # 64, 128,
    n_layer=4,                   #2, 4,
    n_head=2,                   #4,
    pad_token_id=tokenizer.pad_token_id,
)

model = GPT2LMHeadModel(config)

# Проверка количества параметров
n_params = sum(p.numel() for p in model.parameters())
print(f"Model has {n_params} parameters.")  # <1 млн параметров

# Уникальное имя папки для сохранения
timestamp = time.strftime("%Y%m%d-%H%M%S")
output_dir = f"./tiny_gpt2_{timestamp}"

# 7. Аргументы для тренировки
training_args = TrainingArguments(
#    output_dir="./tiny_gpt2",        # Папка для сохранения модели
    output_dir=output_dir,  # Уникальная папка для каждой тренировки
    overwrite_output_dir=True,      # Перезаписать содержимое выходной папки
    save_strategy="epoch",          # Сохранять модель по окончании каждого эпоха
    logging_strategy="steps",
   # log_every_n_steps: Это будет контролировать, как часто данные будут логироваться.
    logging_steps=500,                 #10,            # Логируем каждые 10 шагов
    save_steps=1000,                  # Шаги для сохранения модели #
    save_total_limit=2,             # Максимальное количество сохранений модели
    per_device_train_batch_size=8,      # Размер батча для тренировки
    per_device_eval_batch_size=8,        # Размер батча для валидации
    logging_dir='./logs',                 # Папка для логов
    num_train_epochs=3,                 # Количество эпох
    warmup_steps=500,                      #100,                   # Количество шагов для разогрева
    learning_rate=5e-4,              # Скорость обучения
    weight_decay=0.01,                 # Вес для регуляризации
    report_to="wandb",  # логируем в wandb
    push_to_hub=True,   # автоматическая отправка в Hugging Face
    hub_model_id="hannanechiporenko25/tiny-gpt2-shakespeare",  # свой репозиторий
    hub_strategy="every_save",  # пушить на Hugging Face после каждого сохранения
    logging_first_step=True,          # Логировать первый шаг#
)
#    evaluation_strategy="epoch",      # Сохранять модель по окончании каждого эпоха
# 8. Тренер HuggingFace
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)
# Пример логирования метрик вручную в процессе обучения
#trainer.log_metrics("train", {"loss": loss_value, "learning_rate": lr_value, "grad_norm": grad_norm_value})

# 9. Обучение
trainer.train()

# 10. После тренировки сохраним локально и пушнем
trainer.save_model("./tiny_gpt2_final")
trainer.push_to_hub()
tokenizer.push_to_hub("hannanechiporenko25/tiny-gpt2-shakespeare")

## # 11. Генерация текста после обучения
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Загружаем обученную модель и токенизатор (локально или с Hugging Face Hub)
model = GPT2LMHeadModel.from_pretrained("hannanechiporenko25/tiny-gpt2-shakespeare")
tokenizer = GPT2Tokenizer.from_pretrained("hannanechiporenko25/tiny-gpt2-shakespeare")

model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Стартовая фраза
start_text = "to be or not to be"
input_ids = tokenizer.encode(start_text, return_tensors="pt").to(device)

# Генерация продолжения
output_ids = model.generate(
    input_ids,
    max_length=100,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    pad_token_id=tokenizer.pad_token_id
)

# Вывод результата
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(" Сгенерированный текст:")
print(generated_text)

# from huggingface_hub import upload_file

# upload_file(
#     path_or_fileobj="путь_к_файлу",
#     path_in_repo="hannanechiporenko25/decoder-only-transformer-small",
#     repo_id="ваш_пользователь/название_репозитория",
#     token=os.environ['HF_TOKEN']
# )





  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Li

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


--2025-04-27 14:59:18--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘tinyshakespeare.txt’

tinyshakespeare.txt 100%[===================>]   1.06M  4.28MB/s    in 0.2s    

2025-04-27 14:59:19 (4.28 MB/s) - ‘tinyshakespeare.txt’ saved [1115394/1115394]



Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

...

<ipython-input-5-96564ecf6cda>:120: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
1,9.949300
500,3.302300
1000,0.543300


No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


model.safetensors:   0%|          | 0.00/29.0M [00:00<?, ?B/s]

 Сгенерированный текст:
to be or not to be the have the people.
